In [11]:
import pandas as pd 
import numpy as np
import seaborn as sns 
import matplotlib.pyplot as plt
from IPython.display import display
import warnings

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score

import scipy.stats as stats
import scipy
import scipy.stats.stats
from scipy.stats import poisson

import scipy.stats
import phik

In [12]:
pd.set_option('display.min_rows', 20)
pd.set_option('display.max_rows', 20)
pd.options.display.float_format = '{:,.2f}'.format

import warnings

# Подавление предупреждений SettingWithCopyWarning
warnings.filterwarnings('ignore', category=UserWarning, message='SettingWithCopyWarning')

### Функции

In [13]:
# функция информации по таблице
def dataframe_summary(df, string):
    # Вывод общей информации
    print("Общая информация по таблице:", string)
    df.info()

    # Вывод статистического описания
    print("\n Статистическое описание:")
    display(df.describe().transpose())

    # Вывод случайных примеров
    print("\nСлучайные примеры:")
    display(df.sample(5))

    # Вывод количества строк и столбцов
    print("\nКоличество строк и столбцов:", df.shape)
    
    # Вывод количества явных дубликатов
    print("\nКоличество явных дубликатов:", df.duplicated().sum())
    print('')

In [14]:
# функция поиска пропусков
def analyze_missing_values(dataframes, names):
    for i, df in enumerate(dataframes):
        # Подсчет количества и процента пропущенных значений
        total = df.isnull().sum().sort_values(ascending=False)
        percent = (df.isnull().sum()/df.isnull().count()*100).sort_values(ascending=False)
        missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
        missing_data = missing_data.query('Total > 0')

        # Вывод информации о пропусках
        print(f"Пропуски в датафрейме {names[i]}:\n{missing_data}\n")

        # Создание и отображение тепловой карты
        plt.figure(figsize=(12, 8))
        sns.heatmap(df.isnull(), cbar=False, yticklabels=False)
        plt.title(f"Heatmap пропусков для {names[i]}")
        plt.show()

In [15]:
# функция описательной статистики числового признака
def descriptive_stats_and_plot(df, numeric_column):
    if numeric_column not in df.columns:
        print(f"Столбец '{numeric_column}' не найден в DataFrame.")
        return

    # Описательная статистика
    descriptive_stats = df.groupby('music_genre')[numeric_column].describe()
    display(f"Описательная статистика для {numeric_column}:", descriptive_stats)

    # Визуализация распределения
    plt.figure(figsize=(12, 15))
    sns.violinplot(x=numeric_column, y='music_genre',hue="music_genre" ,data=df, split=True, inner='quart')

    # Расчет общей медианы и среднего
    overall_median = df[numeric_column].median()
    overall_mean = df[numeric_column].mean()

    # Добавление линий медианы и среднего
    plt.axvline(overall_median, color='blue', linestyle='--', alpha=0.7, linewidth=2.5, label=f'Медиана: {overall_median:.2f}')
    plt.axvline(overall_mean, color='red', linestyle='--', alpha=0.7, linewidth=2.5, label=f'Среднее: {overall_mean:.2f}')

    plt.title(f'Распределение {numeric_column} по жанрам музыки')
    plt.legend()
    
    plt.figure(figsize=(12, 15))
    sns.boxplot(x=numeric_column, y='music_genre',hue="music_genre", data=df)

    # Расчет общей медианы и среднего
    overall_median = df[numeric_column].median()
    overall_mean = df[numeric_column].mean()

    # Добавление линий медианы и среднего
    plt.axvline(overall_median, color='blue', linestyle='--', alpha=0.7, linewidth=2.5, label=f'Медиана: {overall_median:.2f}')
    plt.axvline(overall_mean, color='red', linestyle='--', alpha=0.7, linewidth=2.5, label=f'Среднее: {overall_mean:.2f}')

    plt.title(f'Распределение {numeric_column} по жанрам музыки')
    plt.legend()
    plt.show()

In [16]:
def fill_na_with_genre_median(df, numeric_column):
    # Вычисление медианы для каждого жанра
    median_per_genre = df.groupby('music_genre')[numeric_column].median()

    # Функция для заполнения пропусков
    def fill_na(row):
        if pd.isna(row[numeric_column]):
            return median_per_genre[row['music_genre']]
        else:
            return row[numeric_column]

    # Применение функции к каждой строке
    df[numeric_column] = df.apply(fill_na, axis=1)

In [17]:
def fill_na_with_quantile(df, numeric_column):
    # Вычисление 25-го и 75-го квартилей для указанной числовой переменной по каждому жанру
    quantile_table = df.groupby('music_genre')[numeric_column].quantile([0.25, 0.75]).unstack()

    # Переименование колонок
    quantile_table.columns = ['percentile_25', 'percentile_75']

    # Сброс индекса, чтобы 'music_genre' стал колонкой
    quantile_table.reset_index(inplace=True)

    # Объединение с исходным DataFrame
    merged = df.merge(quantile_table, on='music_genre', how='left')

    # Функция для заполнения пропусков
    def fill_na(row):
        if pd.isna(row[numeric_column]):
            return np.random.uniform(row['percentile_25'], row['percentile_75'])
        else:
            return row[numeric_column]

    # Применение функции к каждой строке
    df[numeric_column] = merged.apply(fill_na, axis=1)

In [18]:
# Функция для заполнения пропущенных значений key
def fill_missing_keys(row):
    if pd.isna(row['key']):
        return np.random.choice(top_keys_per_genre[row['music_genre']])
    else:
        return row['key']

In [19]:
def replace_question_marks_with_random(df, column):
    # Преобразование столбца в числовой тип, замена '?' на NaN
    df[column] = pd.to_numeric(df[column], errors='coerce')

    # Вычисление Q1 и Q3 квартилей для указанной переменной по каждому жанру
    quantile_table = df.groupby('music_genre')[column].quantile([0.25, 0.75]).unstack()
    quantile_table.columns = ['Q1', 'Q3']

    # Функция для замены NaN на случайное значение между Q1 и Q3
    def replace_with_random(row):
        if pd.isna(row[column]):
            lower_bound = quantile_table.loc[row['music_genre'], 'Q1']
            upper_bound = quantile_table.loc[row['music_genre'], 'Q3']
            return np.random.uniform(lower_bound, upper_bound)
        else:
            return row[column]

    # Применение функции к каждой строке
    df[column] = df.apply(replace_with_random, axis=1)
    return df

In [20]:
def fill_na_most_frequent_per_genre(df, column, target_column='music_genre'):
    # Вычисление самого частого значения для каждого жанра
    mode_per_genre = df.groupby(target_column)[column].agg(lambda x: x.mode()[0] if not x.mode().empty else np.nan)

    # Функция для заполнения пропусков
    def fill_na(row):
        if pd.isna(row[column]):
            return mode_per_genre[row[target_column]]
        else:
            return row[column]

    # Применение функции к каждой строке
    df[column] = df.apply(fill_na, axis=1)

In [21]:
def categorical_analysis(df, category_column, target_column='music_genre'):
    # Статистическая информация
    stats = df.groupby(target_column)[category_column].value_counts(normalize=True).unstack().fillna(0)
    
    # Графическая информация
    plt.figure(figsize=(12, 6))
    sns.countplot(x=category_column, hue=target_column, data=df)
    plt.title(f'Distribution of {category_column} within {target_column}')
    plt.xticks(rotation=45)
    plt.show()
    
    return stats

In [22]:
file_names = {
    'train': 'train.csv',
    'test': 'test.csv'
}

# Загрузка каждого файла в соответствующую переменную
for variable, file_name in file_names.items():
    globals()[variable] = pd.read_csv(file_name)

file_names = list(file_names.keys())

FileNotFoundError: [Errno 2] No such file or directory: 'train.csv'